In [ ]:
import functools
from sklearn import metrics
import numpy as np
from classes.boosting_matrix import BoostingMatrix
from classes.dataset import Dataset
from settings import Settings
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import Counter
from classes.pattern_boosting import PatternBoosting
from settings import Settings
from classes.enumeration.estimation_type import EstimationType
from data.synthetic_dataset import SyntheticDataset
import pandas as pd
import copy
import matplotlib.pyplot as plt
from classes.analysis_patternboosting import AnalysisPatternBoosting
from data.load_dataset import load_dataset
from data import data_reader
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from multiprocessing.dummy import Pool as ThreadPool

import copy
from data import data_reader
from classes.wrapper_pattern_boosting import WrapperPatternBoosting
from jupiter_notebook_functions import *
from classes.wrapper_pattern_boosting import WrapperPatternBoosting

Settings:

In [ ]:
dataset_name="5_k_selection_graphs"

# the model will be searched in the directory results/jupiter
pattern_boosting_model_name="frequency_matrix"


In [ ]:
# Load the dataset

directory = data_reader.get_save_location(folder_relative_path="results/jupiter", create_unique_subfolder=True)
pattern_boosting = data_reader.load_data(directory=directory, filename="frequency_matrix")

dataset = load_dataset(dataset_name)


# loading the model "pattern_boosting_model_name" and getting the frequency matrix

directory = data_reader.get_save_location(folder_relative_path="results/jupiter", create_unique_subfolder=True)
pattern_boosting = data_reader.load_data(directory=directory, filename=pattern_boosting_model_name)
frequency_matrix = pd.DataFrame(pattern_boosting.boosting_matrix.get_matrix(),
                                columns=pattern_boosting.boosting_matrix.get_header())


# get the frequency matrix

if isinstance(pattern_boosting, PatternBoosting):
    frequency_matrix = pattern_boosting.create_boosting_matrix_for(dataset)

elif isinstance(pattern_boosting, WrapperPatternBoosting):
    frequency_matrix = pattern_boosting.create_ordered_boosting_matrix(dataset)


